In [6]:
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
from functools import partial
import imageio
import cProfile

In [2]:
plt.close('all')

In [3]:
def forward_transform_with_shadow(n, xrange, object_array, reflectivity_array, field_of_view, wavelength, samplerate):
    Object = object_array
    Reflectivity = reflectivity_array
    N = np.size(Reflectivity)
    FOV = field_of_view
    l = wavelength
    sample_rate = samplerate
    
    ## Declaring variables
    newx = xrange
    OneD_transform = np.empty(N,dtype=np.complex)
    OneD_forward_integrand = np.empty(N,dtype=np.complex)
    deltak = 1/FOV
    heightdiff = np.diff(Object)
    h = np.where(heightdiff != 0)[0]
    shadow_function = np.ones(N)
    
    ## Making the shadow function
    if n > 0:
        theta = 2*np.arcsin(n*wavelength*deltak/2)
        d = heightdiff*np.tan(theta/2)
        for i in range(np.size(h)):
            dindex = int(np.round(d[h[i]]/sample_rate))
            if heightdiff[h[i]] > 0:
                if i == 0:
                    shadow_function[max(0,h[i]-dindex):h[i]+1] = 0.0
                elif h[i]-dindex > h[i-1]:
                    shadow_function[max(0,h[i]-dindex):h[i]+1] = 0.0
                elif h[i]-dindex < h[i-1]:
                    if heightdiff[h[i-1]] > 0:
                        dindex = int(np.round((heightdiff[h[i]]+heightdiff[h[i-1]])*np.tan(theta/2)/sample_rate))
                        shadow_function[max(0,h[i]-dindex):h[i]+1] = 0.0 
                    elif heightdiff[h[i-1]] < 0:
                        shadow_function[h[i-1]:h[i]+1] = 0.0
            elif heightdiff[h[i]] < 0:
                if i == np.size(h)-1:
                    shadow_function[h[i]:min(N,h[i]-dindex+1)] = 0.0
                elif h[i]-dindex < h[i+1]:
                    shadow_function[h[i]:min(N,h[i]-dindex+1)] = 0.0
                elif h[i]-dindex > h[i+1]:
                    if heightdiff[h[i+1]] > 0:
                        shadow_function[h[i]:h[i+1]+1] = 0.0
                    elif heightdiff[h[i+1]] < 0:
                        dindex = int(np.round((heightdiff[h[i]]+heightdiff[h[i+1]])*np.tan(theta/2)/sample_rate))
                        shadow_function[h[i]:min(N,h[i]-dindex+1)] = 0.0


    ## forward transform with shadow function applied
    OneD_forward_integrand = Reflectivity*np.exp(n*(-1j)*newx*2*np.pi/N)*shadow_function
    OneD_transform = sum(OneD_forward_integrand)

    
    return OneD_transform

## for use in later cells
if __name__=="__main__":
    p = Pool()
#https://stackoverflow.com/questions/45862974/multiprocessing-simple-function-doesnt-work-but-why

In [56]:
l = 532e-9
FOV = 5e-4 #field of view
max_N = int(np.floor(np.sin(np.pi/2)*2*FOV/l))
images = []

for N in range(50,max_N,10):

    ## setting up the range and the field of view (FOV)
    x = np.linspace(0,FOV,N)
    x_range = np.arange(N)
    sample_rate = x[2] - x[1]

    ## Setting up the reflectivity as a function of x. This is what we'll be taking the transform of.
    Reflectivity = np.ones(N)*(6/10)
    Reflectivity[int(round(19*N/49)):int(round(30*N/49))] = 0.8
    Object = np.ones(N)*(6/10)*FOV
    Object[int(round(19*N/49)):int(round(30*N/49))] = 0.84*FOV
    Object[int(round(22*N/49)):int(round(23*N/49))] = 0.85*FOV
    Object[int(round(24*N/49)):int(round(25*N/49))] = 0.85*FOV
    Object[int(round(26*N/49)):int(round(27*N/49))] = 0.85*FOV

    ## Mapping the trasnform function to a simple array for optimization purpposes
    IPSII = np.asarray(p.map(partial(forward_transform_with_shadow, xrange = x_range, object_array=Object, reflectivity_array=Reflectivity, field_of_view=FOV, wavelength=l, samplerate=sample_rate),range(N)))

    tolerance = 1e-10
    IPSII.real[abs(IPSII.real) < tolerance] = 0.0
    IPSII.imag[abs(IPSII.imag) < tolerance] = 0.0

    ## Inputting the text for the 4th quadrant of the figure
    myfig = plt.figure(figsize=(13,8))
    theta = round(np.arcsin(N*l/(2*FOV))*360/(2*np.pi),2)    # I think this theta is the angle
    plt.gcf().text(0.55, 0.24, 'number of points = '+str(N)+ # from the normal to one of the
                   '\n'+u'\u03B8'+' = '+str(theta)+chr(176), # beams, assuming they are
                   fontsize=14)                              # symmetric accross the normal.
    plt.gcf().text(0.79, 0.295,u'\u03B8',fontsize=20) #theta symbol
    
    ## Just plotting for the next few segments
    plt.subplot(2,2,1)
    plt.title("Object", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("height (m)", fontsize=15)
    plt.axis([-0.00002,0.00052,0.58*FOV,0.9*FOV])
    plt.plot(x,Object)

    plt.subplot(2,2,2)
    plt.title("Original Reflectivity", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("reflectivity (0-1)", fontsize=15)
    plt.axis([-0.00002,0.00052,0.58,0.9])
    plt.plot(x,Reflectivity)

    plt.subplot(2,2,3)
    plt.title("Reflectivity with shadow function applied", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("reflectivity (0-1)", fontsize=15)
    plt.axis([-0.00002,0.00052,-0.05,1.2])
    plt.plot(x,np.fft.ifft((IPSII)).real)
    
    plt.subplot(2,2,4)
    plt.plot(x,x-x[int(round(N/2))],color='black',linewidth=1.5)
    plt.plot(x,-x+x[int(round(N/2))],color='black',linewidth=1.5)
    plt.axvline(x=x[int(round(N/2))],color='black',linewidth=1.5,linestyle='--')
    plt.axis('off')
    plt.tight_layout()
    
    myfig.savefig("/home/carter/Documents/Research/plotfig.png")
    plt.close('all')
    images.append(imageio.imread("/home/carter/Documents/Research/plotfig.png"))
    
imageio.mimsave('/home/carter/Documents/Research/IPSII bumps on top of object.gif', images, duration=0.08)



In [51]:
l = 532e-9
FOV = 5e-4 #field of view
max_N = int(np.floor(np.sin(np.pi/2)*2*FOV/l))
images = []

for N in range(50,max_N,10):

    ## setting up the range and the field of view (FOV)
    x = np.linspace(0,FOV,N)
    x_range = np.arange(N)
    sample_rate = x[2] - x[1]

    ## Setting up the reflectivity as a function of x. This is what we'll be taking the transform of.
    Object = np.ones(N)*(5/10)
    Object[int(round(2*N/10)):int(round(3*N/10))] = 0.8
    Object[int(round(4*N/10)):int(round(5*N/10))] = 0.8
    Object[int(round(6*N/10)):int(round(7*N/10))] = 0.8
    Object = Object*FOV
    Reflectivity = np.ones(N)*(5/10)

    ## Mapping the trasnform function to a simple array for optimization purpposes
    IPSII = np.asarray(p.map(partial(forward_transform_with_shadow, xrange = x_range, object_array=Object, reflectivity_array=Reflectivity, field_of_view=FOV, wavelength=l, samplerate=sample_rate),range(N)))

    tolerance = 1e-10
    IPSII.real[abs(IPSII.real) < tolerance] = 0.0
    IPSII.imag[abs(IPSII.imag) < tolerance] = 0.0


    ## Inputting the text for the 4th quadrant of the figure
    myfig = plt.figure(figsize=(13,8))
    theta = round(np.arcsin(N*l/(2*FOV))*360/(2*np.pi),2)    # I think this theta is the angle
    plt.gcf().text(0.55, 0.24, 'number of points = '+str(N)+ # from the normal to one of the
                   '\n'+u'\u03B8'+' = '+str(theta)+chr(176), # beams, assuming they are
                   fontsize=14)                              # symmetric accross the normal.
    plt.gcf().text(0.79, 0.295,u'\u03B8',fontsize=20) #theta symbol
    
    ## Just plotting for the next few segments
    plt.subplot(2,2,1)
    plt.title("Object", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("height (m)", fontsize=15)
    plt.axis([-0.00002,0.00052,0.000241,0.000408])
    plt.plot(x,Object)

    plt.subplot(2,2,2)
    plt.title("Original Reflectivity", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("reflectivity (0-1)", fontsize=15)
    plt.axis([-0.00002,0.00052,0.48,0.52])
    plt.plot(x,Reflectivity)

    plt.subplot(2,2,3)
    plt.title("Reflectivity with shadow function applied", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("reflectivity (0-1)", fontsize=15)
    plt.axis([-0.00002,0.00052,0.21,0.75])
    plt.plot(x,np.fft.ifft((IPSII)).real)
    
    plt.subplot(2,2,4)
    plt.plot(x,x-x[int(round(N/2))],color='black',linewidth=1.5)
    plt.plot(x,-x+x[int(round(N/2))],color='black',linewidth=1.5)
    plt.axvline(x=x[int(round(N/2))],color='black',linewidth=1.5,linestyle='--')
    plt.axis('off')
    plt.tight_layout()
    
    myfig.savefig("/home/carter/Documents/Research/plotfig.png")
    plt.close('all')
    images.append(imageio.imread("/home/carter/Documents/Research/plotfig.png"))
    
imageio.mimsave('/home/carter/Documents/Research/IPSII three towers.gif', images, duration=0.08)

In [52]:
l = 532e-9
FOV = 5e-4 #field of view
max_N = int(np.floor(np.sin(np.pi/2)*2*FOV/l))
images = []

for N in range(50,max_N,10):

    ## setting up the range and the field of view (FOV)
    x = np.linspace(0,FOV,N)
    x_range = np.arange(N)
    sample_rate = x[2] - x[1]

    ## Setting up the reflectivity as a function of x. This is what we'll be taking the transform of.
    Reflectivity = np.ones(N)*(4/10)
    Reflectivity[int(round(19*N/49)):int(round(30*N/49))] = 0.8
    Object = Reflectivity*FOV
    Reflectivity[int(round(22*N/49)):int(round(23*N/49))] = 0.82
    Reflectivity[int(round(24*N/49)):int(round(25*N/49))] = 0.82
    Reflectivity[int(round(26*N/49)):int(round(27*N/49))] = 0.82

    ## Mapping the trasnform function to a simple array for optimization purpposes
    IPSII = np.asarray(p.map(partial(forward_transform_with_shadow, xrange = x_range, object_array=Object, reflectivity_array=Reflectivity, field_of_view=FOV, wavelength=l, samplerate=sample_rate),range(N)))

    tolerance = 1e-10
    IPSII.real[abs(IPSII.real) < tolerance] = 0.0
    IPSII.imag[abs(IPSII.imag) < tolerance] = 0.0


    ## Inputting the text for the 4th quadrant of the figure
    myfig = plt.figure(figsize=(13,8))
    theta = round(np.arcsin(N*l/(2*FOV))*360/(2*np.pi),2)    # I think this theta is the angle
    plt.gcf().text(0.55, 0.24, 'number of points = '+str(N)+ # from the normal to one of the
                   '\n'+u'\u03B8'+' = '+str(theta)+chr(176), # beams, assuming they are
                   fontsize=14)                              # symmetric accross the normal.
    plt.gcf().text(0.79, 0.295,u'\u03B8',fontsize=20) #theta symbol
    
    ## Just plotting for the next few segments
    plt.subplot(2,2,1)
    plt.title("Object", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("height (m)", fontsize=15)
    plt.axis([-0.00002,0.00052,0.00019,0.00041])
    plt.plot(x,Object)

    plt.subplot(2,2,2)
    plt.title("Original Reflectivity", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("reflectivity (0-1)", fontsize=15)
    plt.axis([-0.00002,0.00052,0.38,0.84])
    plt.plot(x,Reflectivity)

    plt.subplot(2,2,3)
    plt.title("Reflectivity with shadow function applied", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("reflectivity (0-1)", fontsize=15)
    plt.axis([-0.00002,0.00052,0.17,1.03])
    plt.plot(x,np.fft.ifft((IPSII)).real)
    
    plt.subplot(2,2,4)
    plt.plot(x,x-x[int(round(N/2))],color='black',linewidth=1.5)
    plt.plot(x,-x+x[int(round(N/2))],color='black',linewidth=1.5)
    plt.axvline(x=x[int(round(N/2))],color='black',linewidth=1.5,linestyle='--')
    plt.axis('off')
    plt.tight_layout()
    
    myfig.savefig("/home/carter/Documents/Research/plotfig.png")
    plt.close('all')
    images.append(imageio.imread("/home/carter/Documents/Research/plotfig.png"))
    
imageio.mimsave('/home/carter/Documents/Research/IPSII bumps on top of reflectivity.gif', images, duration=0.08)

In [53]:
l = 532e-9
FOV = 5e-4 #field of view
max_N = int(np.floor(np.sin(np.pi/2)*2*FOV/l))
images = []

for N in range(50,max_N,10):

    ## setting up the range and the field of view (FOV)
    FOV = 5e-4
    x = np.linspace(0,FOV,N)
    x_range = np.arange(N)
    sample_rate = x[2] - x[1]
    l = 532e-9

    ## Setting up the reflectivity as a function of x. This is what we'll be taking the transform of.
    Reflectivity = np.ones(N)*(5/10)
    Reflectivity[int(round(19*N/49)):int(round(30*N/49))] = 0.8
    Object = Reflectivity*FOV
    Reflectivity[int(round(22*N/49)):int(round(23*N/49))] = 0
    Reflectivity[int(round(24*N/49)):int(round(25*N/49))] = 0
    Reflectivity[int(round(26*N/49)):int(round(27*N/49))] = 0


    ## Mapping the trasnform function to a simple array for optimization purpposes
    IPSII = np.asarray(p.map(partial(forward_transform_with_shadow, xrange = x_range, object_array=Object, reflectivity_array=Reflectivity, field_of_view=FOV, wavelength=l, samplerate=sample_rate),range(N)))

    tolerance = 1e-10
    IPSII.real[abs(IPSII.real) < tolerance] = 0.0
    IPSII.imag[abs(IPSII.imag) < tolerance] = 0.0


    ## Inputting the text for the 4th quadrant of the figure
    myfig = plt.figure(figsize=(13,8))
    theta = round(np.arcsin(N*l/(2*FOV))*360/(2*np.pi),2)    # I think this theta is the angle
    plt.gcf().text(0.55, 0.24, 'number of points = '+str(N)+ # from the normal to one of the
                   '\n'+u'\u03B8'+' = '+str(theta)+chr(176), # beams, assuming they are
                   fontsize=14)                              # symmetric accross the normal.
    plt.gcf().text(0.79, 0.295,u'\u03B8',fontsize=20) #theta symbol
    
    ## Just plotting for the next few segments
    plt.subplot(2,2,1)
    plt.title("Object", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("height (m)", fontsize=15)
    plt.axis([-0.00002,0.00052,0.000242,0.000408])
    plt.plot(x,Object)

    plt.subplot(2,2,2)
    plt.title("Original Reflectivity", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("reflectivity (0-1)", fontsize=15)
    plt.axis([-0.00002,0.00052,-0.04,0.84])
    plt.plot(x,Reflectivity)

    plt.subplot(2,2,3)
    plt.title("Reflectivity with shadow function applied", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("reflectivity (0-1)", fontsize=15)
    plt.axis([-0.00002,0.00052,0.16,1.1])
    plt.plot(x,np.fft.ifft((IPSII)).real)
    
    plt.subplot(2,2,4)
    plt.plot(x,x-x[int(round(N/2))],color='black',linewidth=1.5)
    plt.plot(x,-x+x[int(round(N/2))],color='black',linewidth=1.5)
    plt.axvline(x=x[int(round(N/2))],color='black',linewidth=1.5,linestyle='--')
    plt.axis('off')
    plt.tight_layout()
    
    myfig.savefig("/home/carter/Documents/Research/plotfig.png")
    plt.close('all')
    images.append(imageio.imread("/home/carter/Documents/Research/plotfig.png"))
    
imageio.mimsave('/home/carter/Documents/Research/IPSII striped object.gif', images, duration=0.08)

In [55]:
l = 532e-9
FOV = 5e-4 #field of view
max_N = int(np.floor(np.sin(np.pi/2)*2*FOV/l))
images = []

for N in range(50,max_N,10):

    ## setting up the range and the field of view (FOV)
    FOV = 5e-4
    x = np.linspace(0,FOV,N)
    x_range = np.arange(N)
    sample_rate = x[2] - x[1]
    l = 532e-9

    ## Setting up the reflectivity as a function of x. This is what we'll be taking the transform of.
    Object = np.ones(N)*(5/10)*FOV
    Object[int(np.floor(19*N/49)):int(np.ceil(30*N/49))] = 0.8*FOV
    Reflectivity = Object/FOV
    Object[int(np.floor(22*N/49)):int(np.floor(23*N/49))] = 0
    Object[int(np.floor(24*N/49)):int(np.floor(25*N/49))] = 0
    Object[int(np.floor(26*N/49)):int(np.floor(27*N/49))] = 0


    ## Mapping the trasnform function to a simple array for optimization purpposes
    IPSII = np.asarray(p.map(partial(forward_transform_with_shadow, xrange = x_range, object_array=Object, reflectivity_array=Reflectivity, field_of_view=FOV, wavelength=l, samplerate=sample_rate),range(N)))

    tolerance = 1e-10
    IPSII.real[abs(IPSII.real) < tolerance] = 0.0
    IPSII.imag[abs(IPSII.imag) < tolerance] = 0.0


    ## Inputting the text for the 4th quadrant of the figure
    myfig = plt.figure(figsize=(13,8))
    theta = round(np.arcsin(N*l/(2*FOV))*360/(2*np.pi),2)    # I think this theta is the angle
    plt.gcf().text(0.55, 0.24, 'number of points = '+str(N)+ # from the normal to one of the
                   '\n'+u'\u03B8'+' = '+str(theta)+chr(176), # beams, assuming they are
                   fontsize=14)                              # symmetric accross the normal.
    plt.gcf().text(0.79, 0.295,u'\u03B8',fontsize=20) #theta symbol
    
    ## Just plotting for the next few segments
    plt.subplot(2,2,1)
    plt.title("Object", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("height (m)", fontsize=15)
    plt.axis([-0.00002,0.00052,-0.00002,0.00041])
    plt.plot(x,Object)

    plt.subplot(2,2,2)
    plt.title("Original Reflectivity", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("reflectivity (0-1)", fontsize=15)
    plt.axis([-0.00002,0.00052,0.48,0.82])
    plt.plot(x,Reflectivity)

    plt.subplot(2,2,3)
    plt.title("Reflectivity with shadow function applied", fontsize=18)
    plt.xlabel("distance (m)", fontsize=15)
    plt.ylabel("reflectivity (0-1)", fontsize=15)
    plt.axis([-0.00002,0.00052,0.16,1.24])
    plt.plot(x,np.fft.ifft((IPSII)).real)
    
    plt.subplot(2,2,4)
    plt.plot(x,x-x[int(round(N/2))],color='black',linewidth=1.5)
    plt.plot(x,-x+x[int(round(N/2))],color='black',linewidth=1.5)
    plt.axvline(x=x[int(round(N/2))],color='black',linewidth=1.5,linestyle='--')
    plt.axis('off')
    plt.tight_layout()
    
    myfig.savefig("/home/carter/Documents/Research/plotfig.png")
    plt.close('all')
    images.append(imageio.imread("/home/carter/Documents/Research/plotfig.png"))
    
imageio.mimsave('/home/carter/Documents/Research/IPSII pits in object.gif', images, duration=0.08)